In [1]:
import os
import osgeo
from osgeo import gdal, ogr, osr
from gdalconst import GA_ReadOnly
from IPython.display import clear_output
import itertools
from multiprocessing import Pool, cpu_count
import time

#my things
from mosaic_rasters_functions import raster_mask_to_shapefile, clip_by_utm_zone

gdal.UseExceptions()

In [2]:
zones = list(range(7, 12)) #12 because python needs the + 1
raster_locations = [os.path.join("..", "data", "non_overlapping_masks", "UTM" + str(zone) + "S_vld_ext.dat") for zone in zones]
save_name = ["UTM_" + str(zone) + "S_mask.shp" for zone in zones]

mask_inputs = [(raster_locations[i], save_name[i]) for i in range(0, len(raster_locations))]

In [3]:
start_time = time.time()
if __name__ == '__main__':
    with Pool() as pool:
        pool.starmap(raster_mask_to_shapefile, mask_inputs)
end_time = time.time()
print("Time to generate shapefile masks")
print(end_time - start_time)

Time to generate shapefile masks
12.141518592834473


Get the extent of BC to merge the rasters together (currently not used due to time procesing times)

In [4]:
bc_boundary = os.path.join("..", "data", "shapefiles", "bc_boundary.shp")

in_driver = ogr.GetDriverByName("ESRI Shapefile")
data = in_driver.Open(bc_boundary, 0)
layer = data.GetLayer()
for feature in layer:
    bc_extent = feature.GetGeometryRef().GetEnvelope()

BC Extent

In [5]:
# Get a Layer's Extent
inShapefile = bc_boundary
inDriver = ogr.GetDriverByName("ESRI Shapefile")
inDataSource = inDriver.Open(inShapefile, 0)
inLayer = inDataSource.GetLayer()
extent = inLayer.GetExtent()

# Create a Polygon from the extent tuple
ring = ogr.Geometry(ogr.wkbLinearRing)
ring.AddPoint(extent[0],extent[2])
ring.AddPoint(extent[1], extent[2])
ring.AddPoint(extent[1], extent[3])
ring.AddPoint(extent[0], extent[3])
ring.AddPoint(extent[0],extent[2])
poly = ogr.Geometry(ogr.wkbPolygon)
poly.AddGeometry(ring)

# Save extent to a new Shapefile
outShapefile = os.path.join("..", "data", "shapefiles", "bc_extent.shp")
outDriver = ogr.GetDriverByName("ESRI Shapefile")

# Remove output shapefile if it already exists
if os.path.exists(outShapefile):
    outDriver.DeleteDataSource(outShapefile)

# Create the output shapefile
outDataSource = outDriver.CreateDataSource(outShapefile)
outLayer = outDataSource.CreateLayer("bc_extent", geom_type=ogr.wkbPolygon)

# Add an ID field
idField = ogr.FieldDefn("id", ogr.OFTInteger)
outLayer.CreateField(idField)

# Create the feature and set values
featureDefn = outLayer.GetLayerDefn()
feature = ogr.Feature(featureDefn)
feature.SetGeometry(poly)
feature.SetField("id", 1)
outLayer.CreateFeature(feature)
feature = None

# Save and close DataSource
inDataSource = None
outDataSource = None

spatialRef = osr.SpatialReference()
spatialRef.ImportFromEPSG(3005)

spatialRef.MorphToESRI()
file = open(os.path.join("..", "data", "shapefiles", 'bc_extent.prj'), 'w')
file.write(spatialRef.ExportToWkt())
file.close()

BC Convex

In [6]:
# Get a Layer
inShapefile = bc_boundary
inDriver = ogr.GetDriverByName("ESRI Shapefile")
inDataSource = inDriver.Open(inShapefile, 0)
inLayer = inDataSource.GetLayer()

# Collect all Geometry
geomcol = ogr.Geometry(ogr.wkbGeometryCollection)
for feature in inLayer:
    geomcol.AddGeometry(feature.GetGeometryRef())

# Calculate convex hull
convexhull = geomcol.ConvexHull()

# Save extent to a new Shapefile
outShapefile = os.path.join("..", "data", "shapefiles", "bc_convex.shp")
outDriver = ogr.GetDriverByName("ESRI Shapefile")

# Remove output shapefile if it already exists
if os.path.exists(outShapefile):
    outDriver.DeleteDataSource(outShapefile)

# Create the output shapefile
outDataSource = outDriver.CreateDataSource(outShapefile)
outLayer = outDataSource.CreateLayer("bc_convex", geom_type=ogr.wkbPolygon)

# Add an ID field
idField = ogr.FieldDefn("id", ogr.OFTInteger)
outLayer.CreateField(idField)

# Create the feature and set values
featureDefn = outLayer.GetLayerDefn()
feature = ogr.Feature(featureDefn)
feature.SetGeometry(convexhull)
feature.SetField("id", 1)
outLayer.CreateFeature(feature)
feature = None

# Save and close DataSource
inDataSource = None
outDataSource = None

spatialRef = osr.SpatialReference()
spatialRef.ImportFromEPSG(3005)

spatialRef.MorphToESRI()
file = open(os.path.join("..", "data", "shapefiles", 'bc_convex.prj'), 'w')
file.write(spatialRef.ExportToWkt())
file.close()

In [7]:
structure_variables = ["loreys_height", "total_biomass", "percentage_first_returns_above_2m"]
vlce_variables = ["vlce"]
utm_variables = ["Logic_Rules_Change_Attribution"]

structure_vlce_years = list(range(1984, 2020))
utm_years = list(range(1985, 2019))

structure_years_variables = itertools.product(structure_variables, structure_vlce_years)
vlce_years_variables = itertools.product(vlce_variables, structure_vlce_years)
utm_years_variables = itertools.product(utm_variables, utm_years)

years_variables = [item for item in vlce_years_variables] + [item for item in utm_years_variables] + [item for item in structure_years_variables]

zones = list(range(7, 12)) #12 because python needs the + 1

#print(years_variables)

Generate mask polygons from Txomin's raster masks

Function from Nick Leach with minor edits

Throws a depreciated warnings due to the CRS that I can't figure out how to fix, but as of March 2021, it still works

In [8]:
num_total = len(years_variables)
print(num_total)
num_done = 1

178


First: Clips rasters to the valid zonal pixels based on Txomin's mask.

Then: Merges clipped rasters into a BC wide raster

In [10]:
mask = os.path.join("..", "data", "shapefiles", "bc_convex.shp")

for item in years_variables[num_done - 1:]:
    variable = item[0]
    year = item[1]
    print(str(num_done), "/", str(num_total))
    print(year, variable)
    
    if variable in structure_variables:
        rasters = [os.path.join("H:\\", "Structure", "UTM_" + str(zone) + "S", variable, "UTM_" + str(zone) + "S_" + variable + "_" + str(year) + ".dat") for zone in zones]
        nodata_value = 0
    
    if variable in vlce_variables:
        rasters = [os.path.join("H:\\", "VLCE", "UTM_" + str(zone) + "S", "HMM", "LC_Class_HMM_" + str(zone) + "S_" + str(year) + ".dat") for zone in zones]
        
        nodata_value = 0
        
    if variable in utm_variables:
        rasters = [os.path.join("H:\\", "utm", "UTM_" + str(zone) + "S", "Change_attribution_logic_rules", 
                                variable + "_UTM_" + str(zone) + "S_" + str(year) + ".tif") for zone in zones]
        
        nodata_value = 65535
        
    
    masks = [os.path.join("..", "data", "non_overlapping_masks", "UTM_" + str(zone) + "S_mask.shp") for zone in zones]
    
    map_inputs = [(rasters[i], masks[i]) for i in range(0, len(rasters))]
    
    print("Parallel processing raster clips")
    
    if __name__ == '__main__':
        with Pool() as pool:
            clipped_rasters = pool.starmap(clip_by_utm_zone, map_inputs)

    warped_save_location = os.path.join("H:\\", "Merged", variable, "BC-" + variable + "-" + str(year) + ".tif")
    warp_options = gdal.WarpOptions(format = "GTiff",
                      dstSRS = "EPSG:3005", 
                      xRes = 30, 
                      yRes = 30,
                      resampleAlg = "near",
                      srcNodata = nodata_value,
                      cutlineDSName = mask,
                      cropToCutline = True
                                   )

    print("Warping Together")
    
    gdal.Warp(warped_save_location, clipped_rasters, options = warp_options)
    
    clear_output(True)
    
    num_done += 1
print("Done")

Done


# Greatest Change Year and Attribution - For single temporal layer (no year field)

In [11]:
gcy_variables = ["Greatest_Change_Year", "Change_Attribution"]

num_total = len(gcy_variables)


num_done = 1

In [12]:
mask = os.path.join("..", "data", "shapefiles", "bc_convex.shp")

for variable in gcy_variables[num_done - 1:]:
    start_time = time.time()
    print(str(num_done), "/", str(num_total))
    print(variable)
    
    if variable == "Greatest_Change_Year":
        rasters = [os.path.join("H:\\", "utm", "UTM_" + str(zone) + "S", "Results", "Change_metrics",
                                "SRef_" + str(zone) + "S_" + variable + ".dat") for zone in zones]
        nodata_value = 0
    if variable == "Change_Attribution":
        rasters = [os.path.join("H:\\", "utm", "UTM_" + str(zone) + "S", "Results", "Change_attribution", "Attribution_UTM" + str(zone) + "S_v2" + ".dat") for zone in zones] 
        nodata_value = 0
        
    masks = [os.path.join("..", "data", "non_overlapping_masks", "UTM_" + str(zone) + "S_mask.shp") for zone in zones]
    
    map_inputs = [(rasters[i], masks[i]) for i in range(0, len(rasters))]
    
    print("Parallel processing raster clips")
    
    if __name__ == '__main__':
        with Pool() as pool:
            clipped_rasters = pool.starmap(clip_by_utm_zone, map_inputs)

    warped_save_location = os.path.join("H:\\", "Merged", variable, "BC-" + variable + ".tif")
    warp_options = gdal.WarpOptions(format = "GTiff",
                      dstSRS = "EPSG:3005", 
                      xRes = 30, 
                      yRes = 30,
                      resampleAlg = "near",
                      srcNodata = nodata_value,
                      cutlineDSName = mask,
                      cropToCutline = True
                                   )
    print("Time to start warp:", time.time() - start_time)
    print("Warping Together")
    
    gdal.Warp(warped_save_location, clipped_rasters, options = warp_options)
    
    clear_output(True)
    print("Time to warp fully:", time.time() - start_time)
    
    num_done += 1
print("Done")

Time to warp fully: 637.6040256023407
Done
